In [71]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


In [72]:
def mfcc_feature_extractor(audio, sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T, axis=0)

    return mfccsScaledFeatures

In [73]:
def contrast_feature_extractor(audio, sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate)
    contrastScaled = np.mean(contrast.T, axis=0)

    return contrastScaled

In [74]:
def tonnetz_feature_extractor(audio, sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T, axis=0)

    return tonnetzScaled

In [75]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [76]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

In [77]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

In [78]:
dict = {
"correct":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",

"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",

"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",
],
"wrong":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",

# "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people", 
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer", 
]}


In [79]:
extractedFeatures = []
for i in dict.keys():
    for x in dict[i]:
        add_features(extractedFeatures, x, i)

In [80]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.to_csv('password.csv', index=False)
data.describe().T

(317, 5)


,count,unique,top,freq
mfcc,317,317,"[-488.23578, 103.44409, 9.910593, 33.63195, 6....",1
contrast,317,317,"[23.98104457049824, 14.883558570433792, 19.139...",1
tonnetz,317,317,"[-0.01461258018698804, 0.0065578272256597275, ...",1
chroma,317,317,"[0.5946693, 0.6137347, 0.62787664, 0.614527, 0...",1
class,317,2,correct,219


In [81]:
# data = pd.read_csv('password.csv')
# print(data.shape)
# data.describe().T

In [82]:
features = data.iloc[:, 0:-1]
target = data['class']
target.unique()
# features.tail()

array(['correct', 'wrong'], dtype=object)

In [83]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [84]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [85]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=105)

### creating RandomForest model

In [86]:
# classifier = RandomForestClassifier(n_estimators=100, criterion="entropy")
# classifier.fit(xTrain,yTrain)
# # 97

### creating SVM model

In [87]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)
# # 93

### creating XGBoost model

In [88]:
classifier = XGBClassifier(objective='multi:softmax', num_class=2, eval_metric='mlogloss')
classifier.fit(xTrain, yTrain)
# 97

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=2, num_parallel_tree=None, ...)

In [89]:
preds = classifier.predict(xTest)

In [90]:
print(classification_report(yTest, preds))

              precision    recall  f1-score   support

           0       0.98      0.91      0.95        57
           1       0.81      0.96      0.88        23

    accuracy                           0.93        80
   macro avg       0.90      0.93      0.91        80
weighted avg       0.93      0.93      0.93        80



In [91]:
def prediction(pred):
    if pred == 0:
        print("Correct!")
    elif pred == 1:
        print("Wrong password")

In [92]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar/omar_open_middle_door_1.wav") # open middle door
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

Correct!
[[0.97554237 0.02445763]]


In [93]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad/Recording (57).wav") # unlock the gate
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

Correct!
[[0.9969002  0.00309986]]


In [94]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem/Recording (11).wav") # grant me access
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

Correct!
[[0.9989077  0.00109237]]


In [95]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem")[0:10]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

Wrong password
[[0.0112247 0.9887754]]
Wrong password
[[0.00379526 0.9962048 ]]
Wrong password
[[0.00987206 0.99012786]]
Wrong password
[[0.00982798 0.9901721 ]]
Wrong password
[[0.02686773 0.9731322 ]]
Wrong password
[[0.00919249 0.9908075 ]]
Wrong password
[[0.01172604 0.988274  ]]
Wrong password
[[0.0139264 0.9860736]]
Wrong password
[[0.02210729 0.9778927 ]]
Wrong password
[[0.01334377 0.9866562 ]]


In [96]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

Correct!
[[9.9906534e-01 9.3468511e-04]]
Correct!
[[0.99445796 0.005542  ]]
Correct!
[[0.9981179  0.00188205]]
Correct!
[[0.99079657 0.00920345]]
Correct!
[[0.86801803 0.131982  ]]


In [97]:
# for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people")[0:5]:
#   testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people/{file}") 
#   pred = classifier.predict(testFeatures)
#   prediction(pred)
#   print(classifier.predict_proba(testFeatures))

In [98]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

Wrong password
[[0.01873421 0.98126584]]
Wrong password
[[0.00438406 0.99561596]]
Wrong password
[[0.00279413 0.9972058 ]]
Wrong password
[[7.1744627e-04 9.9928254e-01]]
Wrong password
[[4.334794e-04 9.995665e-01]]


In [99]:
pickle_out = open("password.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()